In [ ]:
! pip install -q tensorflow-model-optimization

     |████████████████████████████████| 213 kB 12.5 MB/s 


In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers

%load_ext tensorboard

In [ ]:
tf.random.set_seed(1234)

#Model trained on MNIST

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(100),
        layers.Dropout(0.5),
        layers.Dense(10),
    ],)

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=5,
  validation_split=0.1,
)

11501568/11490434 [==============================] - 1s 0us/step
Epoch 1/5
1688/1688 [==============================] - 29s 10ms/step - loss: 0.1960 - accuracy: 0.9413 - val_loss: 0.0732 - val_accuracy: 0.9782
Epoch 2/5
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1194 - accuracy: 0.9643 - val_loss: 0.0710 - val_accuracy: 0.9795
Epoch 3/5
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1063 - accuracy: 0.9692 - val_loss: 0.0701 - val_accuracy: 0.9822
Epoch 4/5
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0954 - accuracy: 0.9720 - val_loss: 0.0640 - val_accuracy: 0.9812
Epoch 5/5
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0881 - accuracy: 0.9741 - val_loss: 0.0590 - val_accuracy: 0.9842


In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.9818000197410583
Saved baseline model to: /tmp/tmpkn5a2csq.h5


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 128)       1280      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 128)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 256)         295168    
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 100)               1254500   
                                                        

#Finetune trained model with pruning

In [ ]:
import tensorflow_model_optimization as tfmot
tf.random.set_seed(1234)
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
# pruning_params = {
#       'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0,
#                                                                final_sparsity=0.50,
#                                                                begin_step=0,
#                                                                end_step=end_step)
# }

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# model_for_pruning.summary()

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)


In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
  
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
  5/422 [..............................] - ETA: 12s - loss: 4.8127e-04 - accuracy: 1.0000WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0149s vs `on_train_batch_end` time: 0.0207s). Check your callbacks.


422/422 [==============================] - 13s 26ms/step - loss: 0.0140 - accuracy: 0.9957 - val_loss: 0.0567 - val_accuracy: 0.9883
Epoch 2/2
422/422 [==============================] - 10s 24ms/step - loss: 0.0093 - accuracy: 0.9972 - val_loss: 0.0598 - val_accuracy: 0.9883


In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9818000197410583
Pruned test accuracy: 0.9866999983787537


In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: /tmp/tmpuyr2ln7f.h5


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

INFO:tensorflow:Assets written to: /tmp/tmpdagmq1yi/assets


Saved pruned TFLite model to: /tmp/tmpmru0ps0y.tflite


In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
# print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 5758394.00 bytes
Size of gzipped pruned Keras model: 1854548.00 bytes


In [ ]:
import tensorflow_model_optimization as tfmot
tf.random.set_seed(1234)
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

def apply_pruning_to_dense(layer):
  if isinstance(layer, layers.Dense):
    return tfmot.sparsity.keras.prune_low_magnitude(layer)
  return layer

def apply_pruning_to_conv(layer):
  if isinstance(layer, layers.Conv2D):
    return tfmot.sparsity.keras.prune_low_magnitude(layer)
  return layer

# Define model for pruning.
# pruning_params = {
#       'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.0,
#                                                                final_sparsity=0.50,
#                                                                begin_step=0,
#                                                                end_step=end_step)
# }

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

# model_for_pruning = tf.keras.models.clone_model(
#     base_model,
#     clone_function=apply_pruning_to_dense,
# )


model_for_pruning = prune_low_magnitude(model, clone_function=apply_pruning_to_conv, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# model_for_pruning.summary()

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)


In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
  
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
  5/352 [..............................] - ETA: 15s - loss: 0.0294 - accuracy: 0.9937WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0166s vs `on_train_batch_end` time: 0.0341s). Check your callbacks.


352/352 [==============================] - 18s 41ms/step - loss: 0.0205 - accuracy: 0.9940 - val_loss: 2.2838 - val_accuracy: 0.7402
Epoch 2/2
352/352 [==============================] - 14s 39ms/step - loss: 0.0013 - accuracy: 0.9998 - val_loss: 2.4634 - val_accuracy: 0.7430


In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.7028999924659729
Pruned test accuracy: 0.7271999716758728


In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))



Saved pruned Keras model to: /tmp/tmpxi_73ovo.h5
Size of gzipped baseline Keras model: 5083879.00 bytes
Size of gzipped pruned Keras model: 1634015.00 bytes


#Model trained on CIFAR10


In [ ]:
# Load CIFAR10 dataset
cifar10 = keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

tf.random.set_seed(1234)

# Define the model architecture.
model = keras.Sequential([
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Flatten(),
        layers.Dense(256),
        layers.Dropout(0.5),
        layers.Dense(256),
        layers.Dense(10),
    ],)

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=10,
  validation_split=0.1,
)

170508288/170498071 [==============================] - 57s 0us/step
Epoch 1/10
1407/1407 [==============================] - 21s 14ms/step - loss: 1.4929 - accuracy: 0.4606 - val_loss: 1.2498 - val_accuracy: 0.5534
Epoch 2/10
1407/1407 [==============================] - 20s 14ms/step - loss: 1.1004 - accuracy: 0.6178 - val_loss: 0.9793 - val_accuracy: 0.6630
Epoch 3/10
1407/1407 [==============================] - 20s 14ms/step - loss: 0.9314 - accuracy: 0.6782 - val_loss: 0.9029 - val_accuracy: 0.6942
Epoch 4/10
1407/1407 [==============================] - 20s 14ms/step - loss: 0.8079 - accuracy: 0.7184 - val_loss: 0.9793 - val_accuracy: 0.6674
Epoch 5/10
1407/1407 [==============================] - 20s 14ms/step - loss: 0.7064 - accuracy: 0.7572 - val_loss: 0.8647 - val_accuracy: 0.7096
Epoch 6/10
1407/1407 [==============================] - 20s 14ms/step - loss: 0.6241 - accuracy: 0.7841 - val_loss: 0.9837 - val_accuracy: 0.6962
Epoch 7/10
1407/1407 [==============================] - 

In [ ]:
model.fit(
  train_images,
  train_labels,
  epochs=2,
  validation_split=0.1,
)

Epoch 1/2
1407/1407 [==============================] - 20s 14ms/step - loss: 0.4934 - accuracy: 0.8419 - val_loss: 1.1567 - val_accuracy: 0.7132
Epoch 2/2
1407/1407 [==============================] - 20s 14ms/step - loss: 0.3525 - accuracy: 0.8819 - val_loss: 1.2150 - val_accuracy: 0.7098


In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.7028999924659729
Saved baseline model to: /tmp/tmp4vzafspv.h5


In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
tf.random.set_seed(1234)
# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
# pruning_params = {
#       'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.0,
#                                                                final_sparsity=0.50,
#                                                                begin_step=0,
#                                                                end_step=end_step)
# }

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# model_for_pruning.summary()

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)


In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
  
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
  5/352 [..............................] - ETA: 18s - loss: 0.0348 - accuracy: 0.9891WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0191s vs `on_train_batch_end` time: 0.0384s). Check your callbacks.


352/352 [==============================] - 18s 41ms/step - loss: 0.0334 - accuracy: 0.9899 - val_loss: 1.9175 - val_accuracy: 0.7400
Epoch 2/2
352/352 [==============================] - 14s 39ms/step - loss: 0.0065 - accuracy: 0.9985 - val_loss: 2.1925 - val_accuracy: 0.7390


In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.7028999924659729
Pruned test accuracy: 0.72079998254776


In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: /tmp/tmp7o5vm2x0.h5


In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))

Size of gzipped baseline Keras model: 5083879.00 bytes
Size of gzipped pruned Keras model: 1628451.00 bytes
